In [18]:
import pandas as pd
import os,glob
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append(os.path.expanduser("~") + "/collproc/")
from raw import *
from coll import *
import matplotlib.pyplot as plt

In [19]:
DIR = os.path.expanduser("~") + "/Dropbox/lml/genome_measurements/"
#DIR = "/Users/dc987/genome_2021/data/genome_measurements/"

In [20]:
allgloss = dirwalk(DIR,require='los',omit=['legacy','flag','Test'])

In [21]:
df = pd.DataFrame()
for f in allgloss:
    tmp = fproc(f,dimension='gloss',threshold=0.5)
    if tmp is not None:
        df = pd.concat([df,tmp])

In [22]:
df = df.drop_duplicates(subset=df.columns.difference(['gloss','fname']))

In [23]:
df = df.reset_index(drop=True)

In [24]:
df['mera'] = 'mod'

In [25]:
df['g60'] = df.gloss.map(lambda x: x[1])

# check on some catalog issues

In [26]:
mcb = ["123","1430","1437","1649","1686","1992","214","2182","2505",
       "2639","2654","294","379","4192","4547","5932","879"]

In [27]:
df.rename(columns={"catalog":"catalogmanbrantrial"},inplace=True)

In [28]:
df['catalog'] = [str(item).split("_")[0] for item in df.catalogmanbrantrial]

In [29]:
df = df.loc[df.catalog!='Average']

In [30]:
set(df.catalog.loc[df.catalog.isin(mcb)])

{'123',
 '1430',
 '1437',
 '1686',
 '1992',
 '2182',
 '2505',
 '294',
 '379',
 '4192',
 '879'}

In [31]:
mcsb = ['2081h', '2643a', '286z', '5302r', '5529r']

In [32]:
set(df.catalog.loc[df.catalog.isin(mcsb)])

{'2081h', '2643a', '286z'}

In [33]:
df.loc[df.catalog.str.contains('294_')]

,catalogmanbrantrial,gloss,gloss_str,fname,mera,g60,catalog


In [34]:
#df.loc[(df.fname.str.contains('binder1_2'))][100:155]

In [35]:
#{'4179b', '4199k', '4199l', '4782t', '4973c', '520q', '5440a'} - set(df.catalog)

In [36]:
db = pd.read_pickle("/Users/damoncrockett/Desktop/uv/db.pkl")

In [37]:
w = ['123',
 '1430',
 '1437',
 '1686',
 '1992',
 '2182',
 '2505',
 '294',
 '379',
 '4192',
 '879',
 '5616',
 '2081h',
 '2643a',
 '286z']

In [38]:
mc = df.loc[~df.catalog.isin(w)]
mc = mc.loc[~mc.catalog.str.contains('2631')]
mc = mc.loc[(~mc.catalog.str.contains('4870'))]

In [39]:
set(mc.catalog)-set(db.catalog)-set(['1786',
                                     '183',
                                     '4782t',
                                    ])

{'5447', '5515z', '5516z'}

In [40]:
df['manbran'] = [item.split("_")[1] for item in df.catalogmanbrantrial]

In [41]:
y = list(set(df.loc[(df.fname.str.contains('binder7_'))&(df.manbran=='mm')].catalog))

In [42]:
y

['4308']

In [108]:
df.loc[df.catalog.str.contains('2355')]

,catalogmanbrantrial,gloss,gloss_str,fname,mera,g60,catalog,manbran
3262,2355_kk_0,"[10.09703, 46.50535, 36.02085]","[10.09703, 46.50535, 36.02085]",gloss_dc_binder3_1.xlsx,mod,46.50535,2355,kk
3263,2355_kk_1,"[7.674811, 40.868, 32.11435]","[7.674811, 40.868, 32.11435]",gloss_dc_binder3_1.xlsx,mod,40.86800,2355,kk
3264,2355_kk_2,"[9.755763, 42.79563, 35.94984]","[9.755763, 42.79563, 35.94984]",gloss_dc_binder3_1.xlsx,mod,42.79563,2355,kk


In [116]:
df.loc[(df.fname.str.contains('binder7_'))&(df.manbran=='bp')]

,catalogmanbrantrial,gloss,gloss_str,fname,mera,g60,catalog,manbran
9933,2383_bp_0,"[13.44685, 48.3391, 61.67331]","[13.44685, 48.3391, 61.67331]",gloss_dc_binder7_3.xlsx,mod,48.33910,2383,bp
9934,2383_bp_1,"[12.77071, 48.28072, 62.74969]","[12.77071, 48.28072, 62.74969]",gloss_dc_binder7_3.xlsx,mod,48.28072,2383,bp
9935,2383_bp_2,"[13.16528, 48.45626, 62.2345]","[13.16528, 48.45626, 62.2345]",gloss_dc_binder7_3.xlsx,mod,48.45626,2383,bp


In [115]:
db.loc[db.catalog.str.contains('2883')]

,catalog,man,bran,year,storfor,ia
4271,2883,Kodak,Velox,1957,Not exposed - package open,


In [117]:
dbs = pd.read_csv("/Users/damoncrockett/Dropbox/lml/export_07_11_24.csv", encoding='latin-1')

In [118]:
dbs.loc[(dbs['Brand']=='Ridax')&(dbs['Texture2']=='Smooth')&(dbs['Reflectance2']=='Semi Matte')&(dbs['BaseColor2']=='Cream')].iloc[:,:12]

,PhotoID,Year,DateUncertain,Catalog Number,Secondary Catalog Number,Manufacturer,Brand,Format,Texture2,Reflectance2,BaseColor2,Weight2
352,61468313,1920,1,5411,h,Gevaert,Ridax,Sample book,Smooth,Semi Matte,Cream,Single Weight
368,1349445055,1920,1,5413,g,Gevaert,Ridax,Sample book,Smooth,Semi Matte,Cream,Double Weight
369,-1163395948,1920,1,5413,h,Gevaert,Ridax,Sample book,Smooth,Semi Matte,Cream,Single Weight
388,1321107823,1920,1,5436,a,Gevaert,Ridax,Sample book,Smooth,Semi Matte,Cream,Double Weight
391,-1048895893,1920,1,5436,d,Gevaert,Ridax,Sample book,Smooth,Semi Matte,Cream,Double Weight
1499,1226751047,1940,1,5255,NaN,Gevaert,Ridax,Not exposed - package open,Smooth,Semi Matte,Cream,Single Weight
2955,2145794947,1950,1,5474,a,Gevaert,Ridax,Sample book,Smooth,Semi Matte,Cream,Single Weight


In [144]:
dbs.loc[dbs['Catalog Number']==2081]

,PhotoID,Year,DateUncertain,Catalog Number,Secondary Catalog Number,Manufacturer,Brand,Format,Texture2,Reflectance2,...,LocationBox,LocationBag,Exposure,BackprintingPrimary,Postcards,Toner,Resin_coated?,ProcessName,Link,AATA_ID
4512,291611768,1960,1,2081,a,Ferrania,Vega,Sample book,Luminous,Glossy,...,11.0,20,3.0,NaN,0,NaN,0,\tgelatin silver process,#http://vocab.getty.edu/page/aat/300139114#,300139114.0
4513,-1913800423,1960,1,2081,b,Ferrania,Vega,Sample book,Smooth (silk) luminous,Semi Matte,...,11.0,20,4.0,NaN,0,NaN,0,\tgelatin silver process,#http://vocab.getty.edu/page/aat/300139114#,300139114.0
4514,274071398,1960,1,2081,c,Ferrania,Vega,Sample book,Fine Grained,Faience,...,11.0,20,5.0,NaN,0,NaN,0,\tgelatin silver process,#http://vocab.getty.edu/page/aat/300139114#,300139114.0
4515,1107887652,1960,1,2081,d,Ferrania,Vega,Sample book,Luminous,Millepunti (silkscreen),...,11.0,20,6.0,NaN,0,NaN,0,\tgelatin silver process,#http://vocab.getty.edu/page/aat/300139114#,300139114.0
4516,1107887807,1960,1,2081,e,Ferrania,Gemma,Sample book,Smooth (silk),Semi Matte,...,11.0,20,7.0,NaN,0,NaN,0,\tgelatin silver process,#http://vocab.getty.edu/page/aat/300139114#,300139114.0
4517,-2113968379,1960,1,2081,f,Ferrania,Gemma,Sample book,Bulgaro,Bulgaro,...,11.0,20,8.0,NaN,0,NaN,0,\tgelatin silver process,#http://vocab.getty.edu/page/aat/300139114#,300139114.0
4518,1107888281,1960,1,2081,h,Ferrania,Plurex,Sample book,Luminous,Glossy,...,11.0,20,9.0,NaN,0,NaN,0,\tgelatin silver process,#http://vocab.getty.edu/page/aat/300139114#,300139114.0
4519,1107888521,1960,1,2081,i,Ferrania,Plurex,Sample book,[texture unspecified],Glossy,...,11.0,20,10.0,NaN,0,NaN,0,\tgelatin silver process,#http://vocab.getty.edu/page/aat/300139114#,300139114.0
4520,795329302,1960,1,2081,j,Ferrania,Plurex,Sample book,Luminous,Glossy,...,11.0,20,11.0,NaN,0,NaN,0,\tgelatin silver process,#http://vocab.getty.edu/page/aat/300139114#,300139114.0
4521,1252390708,1960,1,2081,k,Ferrania,Plurex,Sample book,Smooth (silk) luminous,Semi Matte,...,11.0,20,12.0,NaN,0,NaN,0,\tgelatin silver process,#http://vocab.getty.edu/page/aat/300139114#,300139114.0


In [49]:
#dbs.loc[dbs['Catalog Number']==4782]

In [50]:
dbs.loc[dbs['Catalog Number']==17]

,PhotoID,Year,DateUncertain,Catalog Number,Secondary Catalog Number,Manufacturer,Brand,Format,Texture2,Reflectance2,...,LocationBox,LocationBag,Exposure,BackprintingPrimary,Postcards,Toner,Resin_coated?,ProcessName,Link,AATA_ID
5918,17,1974,0,17,a,Agfa Gevaert,Brovira,Sample book,[texture unspecified],Glossy,...,11.0,21,5.0,A09,0,NaN,0,\tgelatin silver process,#http://vocab.getty.edu/page/aat/300139114#,300139114.0
5919,18,1974,0,17,b,Agfa Gevaert,Brovira,Sample book,Fine Grained,Luster,...,11.0,21,6.0,A10,0,NaN,0,\tgelatin silver process,#http://vocab.getty.edu/page/aat/300139114#,300139114.0
5920,19,1974,0,17,c,Agfa Gevaert,Brovira,Sample book,[texture unspecified],Glossy,...,11.0,21,7.0,A09,0,NaN,0,\tgelatin silver process,#http://vocab.getty.edu/page/aat/300139114#,300139114.0
5921,20,1974,0,17,d,Agfa Gevaert,Brovira,Sample book,[texture unspecified],[gloss unspecified],...,11.0,21,8.0,A10,0,NaN,0,\tgelatin silver process,#http://vocab.getty.edu/page/aat/300139114#,300139114.0
5922,21,1974,0,17,e,Agfa Gevaert,Brovira,Sample book,Smooth,Semi Matte,...,11.0,21,9.0,A10,0,NaN,0,\tgelatin silver process,#http://vocab.getty.edu/page/aat/300139114#,300139114.0
5923,648616702,1974,0,17,f,Agfa Gevaert,Brovira,Sample book,Smooth,Matte,...,11.0,21,10.0,A10,0,NaN,0,\tgelatin silver process,#http://vocab.getty.edu/page/aat/300139114#,300139114.0
5924,23,1974,0,17,g,Agfa Gevaert,Brovira,Sample book,[texture unspecified],Silk,...,11.0,21,11.0,A10,0,NaN,0,\tgelatin silver process,#http://vocab.getty.edu/page/aat/300139114#,300139114.0
5925,24,1974,0,17,h,Agfa Gevaert,Brovira,Sample book,Fine Grained,Glossy,...,11.0,21,12.0,A10,0,NaN,0,\tgelatin silver process,#http://vocab.getty.edu/page/aat/300139114#,300139114.0
5926,28,1974,0,17,i,Agfa Gevaert,Record Rapid,Sample book,[texture unspecified],Glossy,...,11.0,21,13.0,A09,0,NaN,0,\tgelatin silver process,#http://vocab.getty.edu/page/aat/300139114#,300139114.0
5927,26,1974,0,17,j,Agfa Gevaert,Record Rapid,Sample book,Smooth,Semi Matte,...,11.0,21,14.0,A10,0,NaN,0,\tgelatin silver process,#http://vocab.getty.edu/page/aat/300139114#,300139114.0


In [51]:
#sorted(set(df.catalog.loc[df.fname.str.contains('box6_1')]))

In [52]:
def find_exact_sequences(catalog_numbers, end_letter):
    from string import ascii_lowercase
    
    # Generate the full sequence up to the specified end letter
    full_sequence = ''.join(ascii_lowercase[:ascii_lowercase.index(end_letter) + 1])
    
    # Create a dictionary to store catalog numbers by their numeric part
    groups = {}
    for catalog in catalog_numbers:
        numeric_part = ''.join(filter(str.isdigit, catalog))
        alphabetic_part = ''.join(filter(str.isalpha, catalog))
        
        if numeric_part and alphabetic_part:
            if numeric_part not in groups:
                groups[numeric_part] = set()
            groups[numeric_part].add(alphabetic_part)
    
    # Filter the groups to only include those with the exact sequence
    exact_sequences = {k: v for k, v in groups.items() if ''.join(sorted(v)) == full_sequence and len(v) == len(full_sequence)}
    
    return exact_sequences

In [53]:
find_exact_sequences(sorted(set(df.catalog.loc[df.fname.str.contains('box6_1')])),'i')

{'2546': {'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i'},
 '2563': {'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i'}}

In [54]:
df.catalog.loc[df.fname.str.contains('box6_1')]

4204     5530a
4205     5530a
4206     5530a
4207     5530b
4208     5530b
         ...  
13484    4792n
13485    4792n
13486    4792o
13487    4792o
13488    4792o
Name: catalog, Length: 924, dtype: object

In [55]:
allx = glob.glob(os.path.join('/Users/damoncrockett/paperbase/src/assets/img/texture/','*.jpg'))

In [56]:
#find_exact_sequences([os.path.basename(item)[:-4] for item in allx],'k')

# texture

In [57]:
xf = pd.read_csv("/Users/damoncrockett/lmlproc/proc/texture/genome_roughness_768_250_501_21_divide.csv")

In [58]:
xf['mstring'] = [os.path.basename(item)[:-4] for item in xf.localpath]

In [59]:
xf['catalog'] = [item.split("_")[0] for item in xf.mstring]

In [60]:
xf['dname'] = [item.split("/")[5] for item in xf.localpath]

In [61]:
mc = xf.loc[~xf.catalog.isin(w)]
mc = mc.loc[~mc.catalog.str.contains('2631')]
mc = mc.loc[(~mc.catalog.str.contains('4870'))]
mc = mc.loc[(~mc.catalog.str.contains('940'))]

In [62]:
set(mc.catalog)-set(db.catalog)-set(['1786',
                                     '183',
                                     '4782t',
                                     '5515z',
                                     '5516z',
                                     '5302r',
                                     '2047',
                                     '2073z2',
                                     '4038',
                                     '4780c',
                                     '4911m',
                                     '5465p',
                                     '5513kl'
                                    ])

{'5529r'}

In [101]:
db.loc[db.catalog.str.contains('2173')]

,catalog,man,bran,year,storfor,ia
214,2173,Darko,Darko Developing Paper,1918,Not exposed - package open,


In [64]:
#set(db.catalog)-set(mc.catalog)

In [98]:
xf.loc[(xf.catalog.str.contains('3202a'))&(xf.catalog.str.startswith('3202a'))]

,localpath,roughness,mstring,catalog,dname
11340,/Volumes/G-DRIVE Thunderbolt 3/genome_texture/...,0.054552,3202a_ka_0,3202a,texture_box3.4_jd
11341,/Volumes/G-DRIVE Thunderbolt 3/genome_texture/...,0.050861,3202a_ka_1,3202a,texture_box3.4_jd
11342,/Volumes/G-DRIVE Thunderbolt 3/genome_texture/...,0.050579,3202a_ka_2,3202a,texture_box3.4_jd
11343,/Volumes/G-DRIVE Thunderbolt 3/genome_texture/...,0.028498,3202a_ka_4,3202a,texture_box3.4_jd


In [84]:
xf.loc[(xf.catalog.str.contains('82'))&(xf.catalog.str.startswith('8'))]

,localpath,roughness,mstring,catalog,dname
683,/Volumes/G-DRIVE Thunderbolt 3/genome_texture/...,0.049489,882_kv_0,882,texture_binder1_jd
684,/Volumes/G-DRIVE Thunderbolt 3/genome_texture/...,0.042620,882_kv_1,882,texture_binder1_jd
685,/Volumes/G-DRIVE Thunderbolt 3/genome_texture/...,0.045019,882_kv_2,882,texture_binder1_jd
2572,/Volumes/G-DRIVE Thunderbolt 3/genome_texture/...,0.016872,82_kk_0,82,texture_binder3_jd
2573,/Volumes/G-DRIVE Thunderbolt 3/genome_texture/...,0.017943,82_kk_1,82,texture_binder3_jd
2574,/Volumes/G-DRIVE Thunderbolt 3/genome_texture/...,0.017828,82_kk_2,82,texture_binder3_jd
9051,/Volumes/G-DRIVE Thunderbolt 3/genome_texture/...,0.072450,82_nn_0,82,texture_box2.2_jd
9052,/Volumes/G-DRIVE Thunderbolt 3/genome_texture/...,0.072683,82_nn_1,82,texture_box2.2_jd
9053,/Volumes/G-DRIVE Thunderbolt 3/genome_texture/...,0.071460,82_nn_2,82,texture_box2.2_jd


In [66]:
#xf.loc[xf.dname.str.contains('binder4_')][255:300]

In [67]:
ncats = xf.loc[(xf.dname.str.contains('binder4_'))&(xf.catalog.str.startswith('9'))]

In [68]:
#set(ncats.catalog.loc[ncats.catalog.str.contains('940')].apply(lambda x: ''.join(filter(str.isalpha, x))))

In [69]:
#set(ncats.catalog.loc[ncats.catalog.str.contains('904')].apply(lambda x: ''.join(filter(str.isalpha, x))))

In [70]:
#db.loc[db.catalog.str.contains('904')]

In [71]:
db['ia'] = db.catalog.apply(lambda x: ''.join(filter(str.isalpha,x)))

In [72]:
db.ia.value_counts()

       2650
b       231
a       230
c       230
d       223
       ... 
nnn       2
ooo       2
ppp       1
qqq       1
rrr       1
Name: ia, Length: 71, dtype: int64

In [73]:
db.loc[(db.man.str.startswith('M'))&(db.bran.str.startswith('M'))].sort_values('catalog')

,catalog,man,bran,year,storfor,ia
1689,1551,Marvel Universal,Marvelbrome,1942,Not exposed - package sealed,


In [74]:
#db.loc[db.catalog.isin(y)]

# Gloss angles

In [9]:
# plt.figure(figsize=(10, 10))
# for gloss in df.gloss[(df.g60 > 30) & (df.g60 < 35)].sample(30):
#     plt.plot([20,60,85],gloss,linewidth=1,color='grey')

# Legacy

In [10]:
legacygloss = dirwalk(DIR,require=['los','egac'])
legacygloss[0]

'/Users/damoncrockett/Dropbox/lml/genome_measurements/genome_legacy/gloss.xlsx'

We will use only this file, with measurements from 2015; Genevieve did a bunch of tests on binder papers in 2018, but none are at the 60 degree angle.

In [11]:
paul = fproc(legacygloss[0],dimension='gloss')

In [12]:
paul = paul[paul['gloss'].apply(lambda x: len(x)) != 11] # we don't have a 60 degree mean for this item

In [13]:
# If there are 12 measurements, we want the 5th; if there are 3, we want the 2nd

paul['g60'] = paul.gloss.map(lambda x: x[4] if len(x)==12 else x[2])

In [14]:
paul = paul[['catalog','fname','g60']]

In [15]:
#paul.catalog.value_counts().loc[lambda x: x > 1]

# STD

In [16]:
allvals = list(df.g60) + list(paul.g60)

In [17]:
g60std = np.std(allvals)
g60std

30.08689691795721

# Enumeration of Cases

This will be different from the thickness case, because here, our trials are spread across the rows, not the columns. That actually makes things more difficult, especially since the sample IDs are not 100% consistent.

#### Catalog extract

In [18]:
df['cat'] = df.catalog.apply(lambda x: parse_sample_id(x)['catalog'])
df['manbran'] = df.catalog.apply(lambda x: parse_sample_id(x)['manbran'])
df['mtrial'] = df.catalog.apply(lambda x: parse_sample_id(x)['mtrial'])

# Mtrial corrections 

I had meant to do this later in the process, but these corrections are very few and easy to make and will make it easier to assess duplicate cases.

In [19]:
df.mtrial[df.catalog=='2484_kb_'] = '0'
df.mtrial[df.catalog=='2484_kb_001'] = '1'
df.mtrial[df.catalog=='2484_kb_002'] = '2'

In [20]:
df.mtrial[df.catalog=='5515y_al_30'] = '3'

In [21]:
df = df[df.catalog!='Average']

In [22]:
df = df.reset_index(drop=True)

In [23]:
pkl(df,'raw.pkl')
pkl(paul,'lraw.pkl')